# 01_Escolha_features_e_pipeline

## 01_import_bibliotecas

In [13]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util

## 02_Carga_de_dados

In [14]:
# Carregar base original
# df = pd.read_excel("01_candidatos.xlsx")

applicants = 'https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/fontes_tratadas/01_candidatos.parquet'

In [15]:
df = pd.read_parquet(applicants)

## 03_Tratamento_datas

In [16]:
# === Tratamento das datas ===
df["data_criacao"] = pd.to_datetime(df["data_criacao"], errors="coerce")
df["data_atualizacao"] = pd.to_datetime(df["data_atualizacao"], errors="coerce")
hoje = pd.Timestamp.today()

df["meses_desde_criacao"] = ((hoje - df["data_criacao"]).dt.days / 30.44).round()
df["meses_desde_atualizacao"] = ((hoje - df["data_atualizacao"]).dt.days / 30.44).round()

def categorizar_meses(meses):
    if pd.isna(meses): return None
    if meses <= 1: return "≤1 mês"
    elif meses <= 3: return "1–3 meses"
    elif meses <= 6: return "3–6 meses"
    elif meses <= 12: return "6–12 meses"
    elif meses <= 24: return "1–2 anos"
    elif meses <= 36: return "2–3 anos"
    else: return "+3 anos"

df["faixa_criacao"] = df["meses_desde_criacao"].apply(categorizar_meses)
df["faixa_atualizacao"] = df["meses_desde_atualizacao"].apply(categorizar_meses)

## 04_Tratamento_Remuneracao

In [17]:
# === Tratamento de remuneração ===
def extrair_remuneracao_ajustada(texto):
    if pd.isna(texto): return None
    texto = str(texto).lower()

    if any(x in texto for x in [" dia", "/dia", "/ dia"]):
        periodicidade = "dia"
    elif any(x in texto for x in ["mês", "mensal", "mes"]):
        periodicidade = "mes"
    elif any(x in texto for x in ["p/hr", "p/h", "/h", "/ h", "hora", "hr", "hor", "h"]):
        periodicidade = "hora"
    else:
        periodicidade = "desconhecida"

    if any(x in texto for x in ["usd", "us$", "dolar", "us", "$"]):
        taxa = 5.0
    elif any(x in texto for x in ["euro", "€"]):
        taxa = 5.5
    else:
        taxa = 1.0

    multiplicador = 1
    if "mil" in texto or "k" in texto:
        multiplicador = 1000

    numeros = re.findall(r'\d+[.,]?\d*', texto)
    if not numeros:
        return None
    valor_str = numeros[0]
    if "," in valor_str and "." not in valor_str:
        valor_str = valor_str.replace(",", ".")
    elif "." in valor_str and "," in valor_str:
        valor_str = valor_str.replace(".", "").replace(",", ".")

    try:
        valor_float = float(valor_str) * multiplicador
    except:
        return None

    if periodicidade == "desconhecida":
        if valor_float < 15:
            valor_float *= 1518
            periodicidade = "mes"
        elif valor_float < 250:
            periodicidade = "hora"
        elif valor_float < 1000:
            periodicidade = "dia"
        else:
            periodicidade = "mes"

    if periodicidade == "hora": valor_mensal = valor_float * 30 * 8
    elif periodicidade == "dia": valor_mensal = valor_float * 30
    else: valor_mensal = valor_float

    return round(valor_mensal * taxa, 2)

df["remuneracao_mensal_brl"] = df["remuneracao"].apply(extrair_remuneracao_ajustada)
media = df["remuneracao_mensal_brl"].mean()
std = df["remuneracao_mensal_brl"].std()
df["remuneracao_zscore"] = (df["remuneracao_mensal_brl"] - media) / std

#df["faixa_remuneracao"] = pd.qcut(df["remuneracao_mensal_brl"], q=5, labels=["Muito Baixa", "Baixa", "Média", "Alta", "Muito Alta"], duplicates="drop")

# Calcular qcut sem labels primeiro
qcut_result, bins = pd.qcut(df["remuneracao_mensal_brl"], q=5, retbins=True, duplicates="drop")

# Criar rótulos compatíveis com o número de faixas detectadas
faixas = ["Muito Baixa", "Baixa", "Média", "Alta", "Muito Alta"]
labels_validas = faixas[:len(bins) - 1]

# Reaplicar qcut com os labels corretos
df["faixa_remuneracao"] = pd.cut(
    df["remuneracao_mensal_brl"],
    bins=bins,
    labels=labels_validas,
    include_lowest=True
)

## 05_Tratamento_objetivo

### 5.1. Limpeza dos textos

In [18]:
def limpar_texto(texto):
    texto = str(texto).lower()
    texto = re.sub(r'[^\w\s]', '', texto)
    return re.sub(r'\s+', ' ', texto).strip()

df['texto_limpo'] = df['objetivo_profissional'].apply(limpar_texto)

### 5.2. Listas expandidas de categorias, subáreas e níveis

In [19]:
categorias = [
    "Tecnologia da Informação", "Engenharia", "Financeiro / Contábil", "Comercial / Negócios",
    "Consultoria / Projetos", "RH / Pessoas", "Marketing / Comunicação", "Jurídico",
    "Educação / Treinamento", "Logística / Suprimentos", "Saúde", "Design / Criação"
]

subareas = [
    "Desenvolvimento Frontend", "Desenvolvimento Backend", "Desenvolvimento Full Stack",
    "Engenharia de Dados", "Análise de Dados", "Infraestrutura de TI", "UX/UI Design",
    "Qualidade de Software / QA", "Gestão de Projetos Ágeis", "Sistemas Corporativos SAP / ERP / CRM",
    "Recrutamento e Seleção", "Controladoria", "Contabilidade", "Vendas", "Gestão de Produtos", "Marketing Digital"
]

niveis = [
    "Estagiário", "Trainee", "Júnior", "Pleno", "Sênior", "Especialista", "Analista",
    "Consultor", "Líder", "Coordenador", "Gerente", "Diretor", "C-Level"
]

### 5.4. Regras manuais simples por palavras-chave

In [20]:

def regras_categoria(texto):
    if any(p in texto for p in ['desenvolvedor', 'dev', 'programador', 'software', 'dados', 'cloud', 'infra']):
        return "Tecnologia da Informação"
    if any(p in texto for p in ['financeiro', 'contábil', 'controladoria']):
        return "Financeiro / Contábil"
    if any(p in texto for p in ['engenheiro', 'engenharia']):
        return "Engenharia"
    if any(p in texto for p in ['scrum', 'product owner', 'projetos', 'agile']):
        return "Consultoria / Projetos"
    if any(p in texto for p in ['recrutamento', 'rh', 'people']):
        return "RH / Pessoas"
    if any(p in texto for p in ['vendas', 'comercial', 'negócios']):
        return "Comercial / Negócios"
    if any(p in texto for p in ['sap', 'erp', 'oracle']):
        return "Tecnologia da Informação"
    return None

def regras_subarea(texto):
    if any(p in texto for p in ['frontend', 'react', 'angular', 'vue']):
        return "Desenvolvimento Frontend"
    if any(p in texto for p in ['backend', 'java', 'node', 'spring', '.net']):
        return "Desenvolvimento Backend"
    if 'full stack' in texto:
        return "Desenvolvimento Full Stack"
    if any(p in texto for p in ['qa', 'teste']):
        return "Qualidade de Software / QA"
    if any(p in texto for p in ['infra', 'rede', 'cloud']):
        return "Infraestrutura de TI"
    if any(p in texto for p in ['dados', 'bi', 'etl']):
        return "Engenharia de Dados"
    if any(p in texto for p in ['ux', 'ui']):
        return "UX/UI Design"
    if any(p in texto for p in ['scrum', 'po', 'kanban']):
        return "Gestão de Projetos Ágeis"
    if any(p in texto for p in ['sap', 'erp', 'crm']):
        return "Sistemas Corporativos SAP / ERP / CRM"
    return None

def regras_nivel(texto):
    if 'estagi' in texto:
        return 'Estagiário'
    if 'trainee' in texto:
        return 'Trainee'
    if 'junior' in texto or 'jr' in texto:
        return 'Júnior'
    if 'pleno' in texto:
        return 'Pleno'
    if 'senior' in texto or 'sênior' in texto:
        return 'Sênior'
    if 'especialista' in texto:
        return 'Especialista'
    if 'analista' in texto:
        return 'Analista'
    if 'consultor' in texto:
        return 'Consultor'
    if 'lider' in texto or 'líder' in texto:
        return 'Líder'
    if 'coordenador' in texto:
        return 'Coordenador'
    if 'gerente' in texto:
        return 'Gerente'
    if 'diretor' in texto:
        return 'Diretor'
    if 'ceo' in texto or 'cto' in texto or 'cfo' in texto:
        return 'C-Level'
    return None


### 5.5. Carregando o modelo de embeddings

In [21]:
modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

### 5.6. Gerando embeddings dos textos e dos rótulos

In [22]:
emb_cargos = modelo.encode(df['texto_limpo'].tolist(), convert_to_tensor=True)
emb_cat = modelo.encode(categorias, convert_to_tensor=True)
emb_sub = modelo.encode(subareas, convert_to_tensor=True)
emb_niv = modelo.encode(niveis, convert_to_tensor=True)

### 5.7. Função de similaridade (threshold reduzido para 0.4)

In [23]:
def classificar_por_similaridade(emb_texto, emb_opcoes, opcoes):
    similaridades = util.cos_sim(emb_texto, emb_opcoes)
    melhor_indice = similaridades.argmax().item()
    melhor_score = similaridades[0, melhor_indice].item()
    return opcoes[melhor_indice] if melhor_score > 0.4 else "Indefinido"


### 5.8. Aplicar classificação híbrida

In [24]:
categorias_pred = []
subareas_pred = []
niveis_pred = []

for i in range(len(df)):
    texto = df['texto_limpo'][i]
    emb = emb_cargos[i].unsqueeze(0)

    cat = regras_categoria(texto) or classificar_por_similaridade(emb, emb_cat, categorias)
    sub = regras_subarea(texto) or classificar_por_similaridade(emb, emb_sub, subareas)
    niv = regras_nivel(texto) or classificar_por_similaridade(emb, emb_niv, niveis)

    categorias_pred.append(cat)
    subareas_pred.append(sub)
    niveis_pred.append(niv)

df['categoria_profissional'] = categorias_pred
df['subarea_profissional'] = subareas_pred
df['nivel_hierarquico'] = niveis_pred

### 5.9. Exportar e mostrar estatísticas

In [25]:
# df.to_excel("classificados_hibrido_embeddings.xlsx", index=False)

total = len(df)
for col in ['categoria_profissional', 'subarea_profissional', 'nivel_hierarquico']:
    indef = (df[col].str.contains('Indefinido')).sum()
    print(f"{col}: {indef} indefinidos ({indef/total:.2%})")

categoria_profissional: 1661 indefinidos (3.91%)
subarea_profissional: 813 indefinidos (1.91%)
nivel_hierarquico: 2949 indefinidos (6.94%)


In [26]:
# === Refinamento pós-classificação ===
import re

def limpar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = str(texto).lower()
    texto = re.sub(r'[^\w\s]', '', texto)
    return re.sub(r'\s+', ' ', texto).strip()

df['texto_limpo'] = df['objetivo_profissional'].apply(limpar_texto)

# Regras de correção
regras_revisao = [
    {"termo": "auto cad", "categoria": "Engenharia", "subarea": "Desenho Técnico", "nivel": "Técnico"},
    {"termo": "autocad", "categoria": "Engenharia", "subarea": "Desenho Técnico", "nivel": "Técnico"},
    {"termo": "segurança", "categoria": "Tecnologia da Informação", "subarea": "Segurança da Informação"},
    {"termo": "devops", "categoria": "Tecnologia da Informação", "subarea": "Infraestrutura de TI"},
    {"termo": "sap", "categoria": "Tecnologia da Informação", "subarea": "Sistemas Corporativos SAP / ERP / CRM"},
    {"termo": "consultor sap", "categoria": "Tecnologia da Informação", "subarea": "Sistemas Corporativos SAP / ERP / CRM"},
]

# Aplicar regras
for regra in regras_revisao:
    idx = df['texto_limpo'].str.contains(regra["termo"], na=False)
    if "categoria" in regra:
        df.loc[idx, 'categoria_profissional'] = regra["categoria"]
    if "subarea" in regra:
        df.loc[idx, 'subarea_profissional'] = regra["subarea"]
    if "nivel" in regra:
        df.loc[idx, 'nivel_hierarquico'] = regra["nivel"]

# Estatísticas finais
total = len(df)
for campo in ['categoria_profissional', 'subarea_profissional', 'nivel_hierarquico']:
    indef = (df[campo].str.contains("Indefinido")).sum()
    print(f"{campo}: {indef} indefinidos ({indef/total:.2%})")

categoria_profissional: 1658 indefinidos (3.90%)
subarea_profissional: 811 indefinidos (1.91%)
nivel_hierarquico: 2949 indefinidos (6.94%)


In [27]:
df.head()

,id_candidato,cv_pt,cv_en,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,...,meses_desde_atualizacao,faixa_criacao,faixa_atualizacao,remuneracao_mensal_brl,remuneracao_zscore,faixa_remuneracao,texto_limpo,categoria_profissional,subarea_profissional,nivel_hierarquico
0,31000,assistente administrativo\n\n\nsantosbatista\n...,,,(11) 97048-2708,,2021-10-11 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,...,45.0,+3 anos,+3 anos,NaN,NaN,NaN,,Jurídico,Vendas,Estagiário
1,31001,formação acadêmica\nensino médio (2º grau) em ...,,,(11) 93723-4396,Analista Administrativo,2021-10-11 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",...,44.0,+3 anos,+3 anos,1900.0,-0.048530,Muito Baixa,analista administrativo,Consultoria / Projetos,Engenharia de Dados,Analista
2,31002,objetivo: área administrativa | financeira\n\n...,,,(11) 92399-9824,Administrativo | Financeiro,2021-10-11 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",...,45.0,+3 anos,+3 anos,3795.0,-0.028021,Muito Baixa,administrativo financeiro,Financeiro / Contábil,Contabilidade,Diretor
3,31003,formação\nensino médio completo\ninformática i...,,,(11) 98100-1727,Área administrativa,2021-10-11 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",...,45.0,+3 anos,+3 anos,1100.0,-0.057187,Muito Baixa,área administrativa,Logística / Suprimentos,Gestão de Produtos,Estagiário
4,31004,última atualização em 09/11/2021\n­ sp\n\nensi...,,,(11) 92517-2678,,2021-10-11 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,...,45.0,+3 anos,+3 anos,NaN,NaN,NaN,,Jurídico,Vendas,Estagiário


## 06_Tratamento_cv_pt

### 6.1 . Cria funcoes

In [28]:
# Regex para contagem de experiências com datas variadas
def contar_periodos_experiencia_tolerante(texto):
    if pd.isna(texto):
        return 0
    texto = texto.lower()
    padrao = re.compile(
        r"(?:\d{1,2}/\d{1,2}/\d{4}|\d{1,2}/\d{4}|"
        r"(?:jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)[./ -]?\d{4})"
        r"\s*(?:a|à|á|até|–|-)?\s*"
        r"(?:\d{1,2}/\d{1,2}/\d{4}|\d{1,2}/\d{4}|"
        r"(?:jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)[./ -]?\d{4}|atual|presente)",
        flags=re.IGNORECASE
    )
    matches = list(padrao.finditer(texto))
    return len(matches)

In [29]:
# Tempo total de experiência estimado (anos entre maior e menor data)
def extrair_tempo_experiencia_completo(texto):
    if pd.isna(texto):
        return None
    texto = texto.lower()
    anos = set()
    padroes = [
        r"(\d{2}/\d{2}/\d{4})",
        r"(\d{2}/\d{4})",
        r"(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)[/. -]*(\d{4})",
        r"(19|20)\d{2}"
    ]
    for padrao in padroes:
        encontrados = re.findall(padrao, texto)
        for e in encontrados:
            if isinstance(e, tuple):
                ano = int(e[-1])
            else:
                partes = re.findall(r"\d{4}", e)
                if partes:
                    ano = int(partes[0])
                else:
                    continue
            if 1900 <= ano <= datetime.now().year:
                anos.add(ano)
    if re.search(r"atual|presente", texto):
        anos.add(datetime.now().year)
    if len(anos) >= 2:
        return max(anos) - min(anos)
    elif len(anos) == 1:
        return datetime.now().year - min(anos)
    return None

In [30]:
# Nível educacional consolidado (mais alto concluído)
def extrair_nivel_educacional_cv(texto):
    if pd.isna(texto):
        return None
    texto = texto.lower()
    if re.search(r'(ensino\s+fundamental|fundamental\s+completo)', texto):
        return 'ensino fundamental'
    elif re.search(r'(ensino\s+m[eé]dio|m[eé]dio\s+completo)', texto):
        return 'ensino médio'
    elif re.search(r'(superior\s+incompleto|gradua[cç][aã]o\s+incompleta|ensino superior incompleto)', texto):
        return 'superior incompleto'
    elif re.search(r'(ensino\s+superior|superior\s+completo|gradua[cç][aã]o\s+completa|formado em|graduado em|bacharel|licenciatura)', texto):
        return 'ensino superior'
    elif re.search(r'(p[óo]s[-\s]?gradua[cç][aã]o|especializa[cç][aã]o|mba|mestrado|doutorado)', texto):
        return 'pós-graduação ou mais'
    return 'não identificado'

def nivel_educacional_completo(nivel_academico, cv_texto):
    if pd.notna(nivel_academico):
        nivel = nivel_academico.lower()
        if "fundamental" in nivel:
            return "ensino fundamental"
        elif "médio" in nivel:
            return "ensino médio"
        elif "incompleto" in nivel:
            return "superior incompleto"
        elif "superior completo" in nivel or "superior" in nivel:
            return "ensino superior"
        elif any(x in nivel for x in ["pós", "mestrado", "doutorado", "mba"]):
            return "pós-graduação ou mais"
    return extrair_nivel_educacional_cv(cv_texto)

def ajustar_nivel_educacional_final(nivel_raw):
    if pd.isna(nivel_raw):
        return None
    nivel_raw = nivel_raw.lower()
    if "pós" in nivel_raw or "mestrado" in nivel_raw or "doutorado" in nivel_raw or "mba" in nivel_raw:
        return "pós-graduação ou mais"
    elif "superior completo" in nivel_raw or "ensino superior completo" in nivel_raw:
        return "ensino superior"
    elif "superior" in nivel_raw and "incompleto" in nivel_raw:
        return "ensino médio"
    elif "ensino médio" in nivel_raw or "médio completo" in nivel_raw:
        return "ensino médio"
    elif "fundamental" in nivel_raw:
        return "ensino fundamental"
    return "não identificado"

In [31]:
# Verifica menção a SAP
def possui_experiencia_sap(texto):
    if pd.isna(texto):
        return False
    return "sap" in texto.lower()

### 6.1 . Aplica_funcoes

In [32]:
df['tempo_experiencia_anos'] = df['cv_pt'].apply(extrair_tempo_experiencia_completo)
df['quantidade_experiencias'] = df['cv_pt'].apply(contar_periodos_experiencia_tolerante)
df['nivel_educacional'] = df.apply(lambda x: nivel_educacional_completo(x['nivel_academico'], x['cv_pt']), axis=1)
df['nivel_educacional_consolidado'] = df['nivel_educacional'].apply(ajustar_nivel_educacional_final)
df['experiencia_sap'] = df['cv_pt'].apply(possui_experiencia_sap)

## 07_Salvando_base

In [33]:
# === Exportar o resultado final ===
df = df[[
    'id_candidato',
    'data_atualizacao',
    'data_criacao',
    'meses_desde_criacao', 'meses_desde_atualizacao',
    'faixa_criacao', 'faixa_atualizacao',
    'remuneracao',
    'remuneracao_mensal_brl', 'remuneracao_zscore', 'faixa_remuneracao',
    'objetivo_profissional',
    'categoria_profissional', 'subarea_profissional', 'nivel_hierarquico',
    'tempo_experiencia_anos',
    'quantidade_experiencias',
    'nivel_educacional',
    'nivel_educacional_consolidado',
    'experiencia_sap'
]]

In [34]:
df.head()

,id_candidato,data_atualizacao,data_criacao,meses_desde_criacao,meses_desde_atualizacao,faixa_criacao,faixa_atualizacao,remuneracao,remuneracao_mensal_brl,remuneracao_zscore,faixa_remuneracao,objetivo_profissional,categoria_profissional,subarea_profissional,nivel_hierarquico,tempo_experiencia_anos,quantidade_experiencias,nivel_educacional,nivel_educacional_consolidado,experiencia_sap
0,31000,2021-10-11 07:29:49,2021-10-11 07:29:49,45.0,45.0,+3 anos,+3 anos,,NaN,NaN,NaN,,Jurídico,Vendas,Estagiário,8.0,8,ensino superior,não identificado,True
1,31001,2021-11-11 11:10:31,2021-10-11 08:56:16,45.0,44.0,+3 anos,+3 anos,1900,1900.0,-0.048530,Muito Baixa,Analista Administrativo,Consultoria / Projetos,Engenharia de Dados,Analista,8.0,0,superior incompleto,ensino médio,False
2,31002,2021-10-11 11:42:36,2021-10-11 09:01:00,45.0,45.0,+3 anos,+3 anos,"2.500,00",3795.0,-0.028021,Muito Baixa,Administrativo | Financeiro,Financeiro / Contábil,Contabilidade,Diretor,13.0,5,ensino superior,não identificado,True
3,31003,2021-10-11 16:04:51,2021-10-11 09:08:13,45.0,45.0,+3 anos,+3 anos,"1100,00",1100.0,-0.057187,Muito Baixa,Área administrativa,Logística / Suprimentos,Gestão de Produtos,Estagiário,14.0,6,superior incompleto,ensino médio,False
4,31004,2021-10-11 09:22:03,2021-10-11 09:18:46,45.0,45.0,+3 anos,+3 anos,,NaN,NaN,NaN,,Jurídico,Vendas,Estagiário,19.0,2,ensino superior,não identificado,False


In [ ]:
df.to_excel("candidatos_tratados_clusterizacao_v2.xlsx", index=False)
print("Arquivo exportado: candidatos_tratados_clusterizacao.xlsx")